In [26]:
from dotenv import load_dotenv
import os
import syncflowai
import json
from MySQLdb import Error
import MySQLdb
import numpy as np
import uuid
load_dotenv()

True

In [9]:
db_name = os.getenv('DATABASE')
host_name = os.getenv('DATABASE_HOST')
user_name = os.getenv('DATABASE_USERNAME')
user_password = os.getenv('DATABASE_PASSWORD')

In [10]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = MySQLdb.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            db=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [11]:
conn = create_connection(host_name, user_name, user_password, db_name)

Connection to MySQL DB successful


In [13]:
mission = syncflowai.generate_mission()
processed_mission, _ = syncflowai.extract_mission_details(mission)
feedback = json.loads(syncflowai.give_feedback(mission, processed_mission['detail']))

In [42]:
print(" ______MISSION______ \n\n", mission, end= "\n\n")
print("______PROCESSED______ \n\n", processed_mission['detail'], end = "\n\n")
print("______FEEDBACK______ \n\n", feedback, end = "\n\n")

 ______MISSION______ 

 I am looking for a skilled individual or team to complete a project with a focus on functionality and performance. You can choose to implement the solution using a combination of 'Ruby', 'Linux', and 'WebRTC' if that aligns with your expertise, or you have the freedom to use any technology stack that you are comfortable with as long as the end product meets the project requirements.

The end goal is to have a robust and scalable web application that offers real-time communication features. It should be secure, responsive, and user-friendly. I will provide detailed specifications regarding the features and functionality that I need, and I expect regular updates and open communication throughout the development process.

Please ensure that the code is clean, well-documented, and maintainable, with an emphasis on quality and security. The application will undergo a thorough review and testing phase, so please plan accordingly for any necessary revisions.

Your expe

In [56]:
json.loads(feedback)['prompt_version']

KeyError: 'prompt_version'

In [20]:
feedback = json.loads(feedback)

In [28]:
feedback['id'] = uuid.uuid1().hex
feedback['prompt_version'] = np.random.choice(['v1', 'v2'])
feedback['mission_id'] = processed_mission['id']

In [29]:
uuid.uuid1().hex

'10a567aa86ad11eeb0c8820394059a04'

In [30]:
def store_feedback(feedback):
    connection = get_db_connection()

    try:
        cursor = connection.cursor()

        # Préparation et exécution de la requête SQL
        insert_query = """
            INSERT INTO user_feedback (
                user_rating, user_comments, prompt_version, mission_id
            ) VALUES (
                %s, %s, %s, %s, %s
            )
            """
        cursor.execute(insert_query, (
            feedback.get("user_rating"), 
            feedback.get("user_comments"),
            feedback.get("prompt_version"),
            feedback.get('mission_id')

        ))

    except MySQLdb.Error as err:
        raise err

    finally:
        # Fermeture de la connexion à la base de données
        if connection and connection.open:
            cursor.close()
            connection.close()